In [1]:
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import copy, time
import random
import pickle
import scipy

import timeit

import mlrfit as mf
import mfmodel as mfm
import numba as nb

from scipy.sparse import coo_matrix
from scipy.linalg import block_diag, pinvh

/Users/parshakova.tanya/opt/anaconda3/envs/routing/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(1001)
random.seed(1001)

# True model is MLR factor model with SNR = 4 

Let $\Sigma = FF^T + D$ be MLR. We generate samples using 
$$
y = Fz + e, \qquad z \sim N(0, I), \qquad N(0, D).
$$

In [3]:
mtype = "small_mlr_hier"
n = 100000
signal_to_noise = 4



L = 5

# ranks = np.array([5, 4, 3, 2, 1])
ranks = np.array([30, 20, 10, 5, 1])
rank = ranks.sum()

nsamples = rank * 4
nsamples

264

In [4]:
pi_rows = np.random.permutation(n)
hpart = {'rows':{'pi':pi_rows, 'lk':[]}, 'cols':{'pi':pi_rows, 'lk':[]}} 
for ngroups in [2, 5, 9, 17, n+1]:
       hpart['rows']['lk'] += [ np.linspace(0, n, ngroups, endpoint=True, dtype=int)]
hpart['rows']['lk'][1] = np.delete(hpart['rows']['lk'][1], -2)
hpart['rows']['lk'][2] = np.delete(hpart['rows']['lk'][2], -4)
hpart['cols']['lk'] = hpart['rows']['lk']
part_sizes = mfm.print_hpart_numgroups(hpart)
mfm.valid_hpart(hpart)

level=0, num_groups=1, mean_size=100000.0
level=1, num_groups=3, mean_size=33333.3
level=2, num_groups=7, mean_size=14285.7
level=3, num_groups=16, mean_size=6250.0
level=4, num_groups=100000, mean_size=1.0


In [5]:
F_hpart = {"pi": hpart['rows']["pi"], "lk": hpart['rows']["lk"][:-1]}
true_mfm = mfm.MFModel()
true_mfm = mfm.generate_mfmodel(true_mfm, n, F_hpart, ranks, signal_to_noise, debug=False)
F_hpart["pi_inv"] = true_mfm.pi_inv

signal_var=81.15506713197232, noise_var=16.242432778950498
SNR=3.9964847160793444, signal_to_noise=4


In [6]:
n, true_mfm.num_factors(), L, ranks.sum()

(100000, 240, 5, 66)

In [7]:
v = np.random.randn(n, 1)


hat_x = true_mfm.solve(v, eps=1e-12, max_iter=20, printing=False)
reldiff = np.linalg.norm(true_mfm.matvec(hat_x) - v) / np.linalg.norm(v)
print(f"solve {reldiff=}")

solve reldiff=9.744615586746802e-13


In [8]:
%timeit true_mfm.solve(v, eps=1e-12, max_iter=20, printing=False)

8.1 s ± 1.86 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
true_mfm.D.min(), true_mfm.D.max()

(0.0006785230567792892, 32.456283388481424)

# EM

In [10]:
C = mfm.sample_data(nsamples, true_mfm)

Z = (C - C.mean(axis=1, keepdims=True))[F_hpart["pi"], :]
# unpermuted_A = (Z @ Z.T / (Z.shape[1]-1))[true_mfm.pi_inv, :][:, true_mfm.pi_inv]

# permute to put clusters on diagonal
Y = Z.T
N = Y.shape[0]

permuted_F_hpart = {"pi_inv":np.arange(n), "pi":np.arange(n), "lk":F_hpart["lk"]}
row_selectors, si_groups, F_hpart, groups_all = mfm.row_col_selections(hpart, return_groups=True)


mfm_Sigma = mfm.MFModel(hpart=F_hpart, ranks=ranks)
mfm_Sigma.init_FD(ranks, F_hpart, init_type="Y", Y=Y)
F0, D0 = mfm_Sigma.F, mfm_Sigma.D

num_levels=5, num_sparsities=16
(16, 4) [ 0  2  6 15]


In [11]:
rank, mfm_Sigma.num_factors()

(66, 240)

In [12]:
loglikelihoods = [-np.inf]
N = Y.shape[0]
eps = 1e-12
for t in range(3):
    F1 = mfm.fast_EM_get_F(F0, D0, Y, ranks, permuted_F_hpart, row_selectors, si_groups)
    D1 = mfm.fast_EM_get_D(F0, D0, F1, Y, ranks, permuted_F_hpart, row_selectors, si_groups)
    F0, D0 = F1, D1
    assert D1.min() >= -1e-8 #and loglikelihoods[-2] - 1e-8 <= loglikelihoods[-1]
    if t % 1 == 0:
        obj = mfm.fast_loglikelihood_value(F0, D0, Y, ranks, permuted_F_hpart, mfm_Sigma.num_factors(),
                                           tol=1e-5)
        loglikelihoods += [obj]
        print(f"{t=}, {obj=},  {D1.min()=}, {D1.max()=}")


print("PASSED")

t=0, obj=-285268.7084904851,  D1.min()=2.804383284514381, D1.max()=66.71522417869596
t=1, obj=-274457.86434678646,  D1.min()=0.3804383003692127, D1.max()=44.27694622515326
t=2, obj=-270917.18312209036,  D1.min()=0.16481087561646057, D1.max()=41.762775694112534
PASSED
